In [1]:
import baltic as bt
import numpy as np
import pandas as pd
import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import re
from matplotlib import gridspec
import scipy.stats as stats
from scipy.stats import gaussian_kde
from matplotlib import rcParams 
# from matplotlib import style
from matplotlib.patches import Patch
from matplotlib.patches import Rectangle
import matplotlib.patheffects as path_effects

# 1. Basic leaf shape stats

In [2]:
palms=pd.read_csv('../data_files/palms_alltraits_curated_20220620.csv',sep='\t')
palms.groupby(by='shape').count()['species'].reset_index()

,shape,species
0,bipinnate,16
1,cospalmate,455
2,entire,108
3,pinnate,1405
4,variable,140


## 1.1. Using the 2550 species phylogeny to quantify data

In [3]:
treeFileNexus='../data_files/Clean_1_1_MCCT_nexus.nex'
cc=bt.loadNexus(treeFileNexus,absoluteTime=False,tip_regex='_([0-9\-]+)$') ## treeFile here can alternatively be a path to a local file

cc.treeStats()
cc.drawTree()
cc.sortBranches()
cc.setAbsoluteTime(0)


Tree height: 108.328870
Tree length: 18144.538310
strictly bifurcating tree
annotations present

Numbers of objects in tree: 5099 (2549 nodes and 2550 leaves)



In [4]:
# list of species in the tree:
sppintree=[k.name for k in cc.getExternal()]
print('Number of species in the tree: %s'%(len(sppintree)))

intree=palms[palms['tip_name'].isin(sppintree)]
print('Number of species remaining in the dataset: %s'%(len(intree)))
print('Number of species not considered in the stats*: %s'%(len(sppintree)-len(intree)))
print('*These are climbing species or species with no information (few)')

Number of species in the tree: 2550
Number of species remaining in the dataset: 2071
Number of species not considered in the stats*: 479
*These are climbing species or species with no information (few)


In [5]:
intree['CHELSA_bio1'].dropna()

0       2.306425
1       2.226600
2       2.407391
3       2.422426
4       2.410777
          ...   
1588    2.021189
1589    2.054996
1590    2.141450
1591    2.435367
1592    2.202761
Name: CHELSA_bio1, Length: 1570, dtype: float64

In [6]:
shapeperc=intree.groupby(by='shape').count()['species'].reset_index().copy(deep=True)
shapeperc['percentage']=['%.2f'%(x) for x in (shapeperc['species']*100)/len(intree)]
shapeperc

,shape,species,percentage
0,bipinnate,14,0.68
1,cospalmate,440,21.25
2,entire,106,5.12
3,pinnate,1370,66.15
4,variable,140,6.76


# 2. Within-species leaf shape variation

In [7]:
intree['shape'].unique()
intree[intree['shape']=='variable']

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,species,coordinateUncertaintyInMeters,decimalLatitude,decimalLongitude,gbifID,individualCount,...,CHELSA_vpd_min_stand,Simard_Pinto_3DGlobalVeg_JGR_stand,MaxStemHeight_m_stand,MaxStemDia_cm_stand,MaxLeafNumber_stand,Max_Blade_Length_m_stand,Max_Rachis_Length_m_stand,Max_Petiole_length_m_stand,StemHeightBladeLength_stand,HeightOverCanopy_stand
195,195,195,195,195,Bactris_acanthocarpa,100.0,-3.950000,-68.847224,1.260222e+09,1.0,...,-0.098457,1.646933,-0.272948,-0.088756,0.374731,1.202122,0.319509,1.920003,0.401766,-1.433035
211,211,211,211,211,Bactris_coloradonis,3615.0,8.631950,-78.959858,1.260244e+09,1.0,...,-0.794410,-0.536819,0.887431,0.019137,-1.219380,0.975348,0.319509,1.152264,0.902899,0.747914
213,213,213,213,213,Bactris_corossilla,100.0,-0.973717,-73.830278,1.258443e+09,1.0,...,-0.329758,2.033555,0.483767,-0.519291,-0.734762,0.733242,-0.072151,-0.696866,0.489443,-1.442500
218,218,218,218,218,Bactris_faucium,NaN,-17.080000,-65.366660,1.259842e+09,1.0,...,0.516724,-0.536819,0.483767,-1.154232,-0.347803,-0.680968,-1.252286,-0.696866,0.263407,0.678671
223,223,223,223,223,Bactris_glandulosa,437.0,8.790000,-83.418789,1.259766e+09,2.0,...,-0.479248,-0.536819,0.346096,-0.431984,-1.219380,0.733242,0.058299,0.961077,0.481993,0.702968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842,1842,2351,2351,443,Hydriastele_boumae,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.868593,1.090952,NaN,0.931468,0.628320,0.158308,1.672763,NaN
1891,1891,2400,2400,492,Iguanura_piahensis,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.272948,NaN,NaN,-0.090097,NaN,-0.696866,-0.270351,NaN
1893,1893,2402,2402,494,Iguanura_sanderiana,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.435778,-0.615804,NaN,0.173153,-0.164402,0.158308,-0.208842,NaN
1896,1896,2405,2405,497,Iguanura_wallichiana,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.183266,-0.615804,-0.347803,0.504407,0.257511,-0.696866,0.220426,NaN


In [8]:
intree[intree['shape']=='variable'][['PalmSubfamily','PalmTribe','tip_name']].to_csv('../data_files/Polymorphic_species_20220706.csv',sep='\t')

# 3. Percentage of annotated species per variable

In [9]:
list(intree.columns)
annotated=intree[[x for x in intree.columns if 'stand' in x]].describe().T['count'].reset_index().copy(deep=True)
annotated['percent']=['%.2f'%(s) for s in annotated['count']*100/len(intree)]
annotated

,index,count,percent
0,CHELSA_ai_stand,1570.0,75.81
1,CHELSA_bio10_stand,1570.0,75.81
2,CHELSA_bio11_stand,1570.0,75.81
3,CHELSA_bio12_stand,1570.0,75.81
4,CHELSA_bio13_stand,1570.0,75.81
5,CHELSA_bio14_stand,1570.0,75.81
6,CHELSA_bio15_stand,1570.0,75.81
7,CHELSA_bio16_stand,1570.0,75.81
8,CHELSA_bio17_stand,1570.0,75.81
9,CHELSA_bio18_stand,1570.0,75.81


In [10]:
# annotated.to_csv('../data_files/variables_sppannotated_20220620.txt',sep='\t')

# 4. Number of climbing species

In [11]:
# list made in the 02_Variables_database notebook
climbing=pd.read_csv('../data_files/climbing_spp_20220620.txt',sep='\t')
# before filtering out species not in the Faurby et al. (2016) tree
climbing.describe()

,Unnamed: 0.1,Unnamed: 0,index,coordinateUncertaintyInMeters,decimalLatitude,decimalLongitude,gbifID,individualCount,year,CHELSA_ai,...,Climbing,Acaulescent,MaxStemHeight_m,MaxStemDia_cm,MaxLeafNumber,Max_Blade_Length_m,Max_Rachis_Length_m,Max_Petiole_length_m,StemHeightBladeLength,HeightOverCanopy
count,525.000000,525.000000,525.000000,91.000000,315.000000,315.000000,3.150000e+02,52.000000,307.000000,315.000000,...,525.000000,525.000000,525.000000,377.000000,0.0,525.000000,296.000000,525.000000,0.0,3.150000e+02
mean,1266.619048,1266.619048,503.419048,2169.265824,4.229816,93.452297,1.452608e+09,1.980769,1975.923453,-80951.303119,...,1.028571,0.007619,16.671429,3.497215,NaN,1.192181,1.620304,0.080162,NaN,1.659505e+11
std,770.521881,770.521881,360.325150,8006.350905,8.840948,57.409357,7.005438e+08,1.734770,31.874957,38923.325672,...,0.166758,0.123325,20.692084,5.273225,NaN,1.314228,1.132431,0.214460,NaN,2.215250e+11
min,337.000000,337.000000,79.000000,0.010000,-28.583697,-96.033315,9.412378e+07,1.000000,1845.000000,-99999.000000,...,1.000000,0.000000,0.000000,0.300000,NaN,0.000000,0.190000,0.000000,NaN,0.000000e+00
25%,515.000000,515.000000,248.000000,1.000000,-0.720695,101.110750,9.124097e+08,1.000000,1972.500000,-99999.000000,...,1.000000,0.000000,0.000000,1.300000,NaN,0.000000,0.800000,0.000000,NaN,6.010345e-01
50%,1102.000000,1102.000000,428.000000,15.500000,4.181667,113.833333,1.258627e+09,1.000000,1982.000000,-99999.000000,...,1.000000,0.000000,10.000000,2.200000,NaN,0.900000,1.300000,0.000000,NaN,1.003000e+11
75%,2088.000000,2088.000000,596.000000,446.500000,9.120818,120.210000,1.930579e+09,2.000000,1997.000000,-99999.000000,...,1.000000,0.000000,25.000000,4.000000,NaN,2.000000,2.162500,0.000000,NaN,2.617000e+11
max,2640.000000,2640.000000,1618.000000,65886.000000,27.206512,153.193425,3.032113e+09,10.000000,2020.000000,3.135691,...,2.000000,2.000000,170.000000,70.000000,NaN,7.000000,6.400000,2.100000,NaN,1.534000e+12


In [12]:
# list of species in the tree:
sppintree=[k.name for k in cc.getExternal()]
print('Number of species in the tree: %s'%(len(sppintree)))

climbintree=climbing[climbing['tip_name'].isin(sppintree)]
print('Number of Climbing species in the tree: %s'%(len(climbintree)))

Number of species in the tree: 2550
Number of Climbing species in the tree: 479


In [13]:
climbintree[['PalmSubfamily','PalmTribe','tip_name']].to_csv('../data_files/Climbingintree_species_20220706.csv',sep='\t')

## 5. Distribution of species medians by variable

In [14]:
variables=['CHELSA_ai','CHELSA_bio1','CHELSA_bio4','CHELSA_bio12','CHELSA_bio15',
           'Max_Rachis_Length_m','StemHeightBladeLength','HeightOverCanopy']

intree.groupby(by='shape')[variables].min().apply(lambda x: 10**x).reset_index()

,shape,CHELSA_ai,CHELSA_bio1,CHELSA_bio4,CHELSA_bio12,CHELSA_bio15,Max_Rachis_Length_m,StemHeightBladeLength,HeightOverCanopy
0,bipinnate,1.0,186.5,181.0,3403.0,141.5,5.50,1.0,8.000000e+10
1,cospalmate,1.0,44.5,109.0,96.0,108.0,1.29,1.0,1.000000e+00
2,entire,1.0,39.0,84.0,161.0,111.5,1.05,1.0,1.000000e+00
3,pinnate,1.0,18.0,92.0,12.0,96.0,1.11,1.0,1.000000e+00
4,variable,1.0,22.0,136.5,145.0,106.0,1.06,1.0,1.000000e+00


In [15]:
intree.groupby(by='shape')[variables].max().apply(lambda x: 10**x).reset_index()

,shape,CHELSA_ai,CHELSA_bio1,CHELSA_bio4,CHELSA_bio12,CHELSA_bio15,Max_Rachis_Length_m,StemHeightBladeLength,HeightOverCanopy
0,bipinnate,100001.548709,272.0,4102.0,28172.5,1115.0,9.0,47.00,4.600000e+11
1,cospalmate,100003.433253,274.5,5608.5,44539.0,1670.0,8.0,48.75,4.710000e+11
2,entire,100002.855431,284.5,6536.5,35492.0,1263.0,5.5,28.80,2.000000e+11
3,pinnate,100003.976216,287.5,6973.0,46257.0,1382.5,19.5,69.12,5.820000e+11
4,variable,100002.408126,281.5,6484.0,32427.0,1240.0,16.0,36.40,1.934000e+11


In [16]:
intree.groupby(by='shape')[variables].median().apply(lambda x: 10**x).reset_index()

,shape,CHELSA_ai,CHELSA_bio1,CHELSA_bio4,CHELSA_bio12,CHELSA_bio15,Max_Rachis_Length_m,StemHeightBladeLength,HeightOverCanopy
0,bipinnate,1.000000,250.959658,1365.191562,7008.107341,623.247543,7.000000,28.622544,2.827366e+11
1,cospalmate,1.000000,211.500000,1474.000000,8227.000000,405.000000,2.400000,7.000000,7.000000e+10
2,entire,1.000000,246.500000,1514.000000,2940.000000,521.000000,1.329962,3.819987,1.350000e+10
3,pinnate,1.000000,257.000000,750.491839,12194.983600,518.499759,2.900000,7.389993,4.064997e+10
4,variable,100000.025091,252.500000,836.000000,12243.000000,481.500000,1.550000,4.614997,1.750000e+10


In [17]:
# tallest entire species
intree[intree['shape']=='entire'][['tip_name','StemHeightBladeLength']].sort_values(by='StemHeightBladeLength', ascending=False).reset_index()

,index,tip_name,StemHeightBladeLength
0,2190,Verschaffeltia_splendida,1.459392
1,1137,Marojejya_darianii,1.322219
2,2031,Phoenicophorium_borsigianum,1.255273
3,1202,Pelagodoxa_henryana,1.231724
4,1468,Sclerosperma_profizianum,1.183270
...,...,...,...
101,1845,Hydriastele_cariosa,0.000000
102,65,Areca_dayung,0.000000
103,2076,Pinanga_jambusana,0.000000
104,209,Bactris_chocoensis,0.000000


In [18]:
list(intree.columns)
intree['dissect'].unique()
intree.groupby(by='dissect')[variables].median().apply(lambda x: 10**x).reset_index()

,dissect,CHELSA_ai,CHELSA_bio1,CHELSA_bio4,CHELSA_bio12,CHELSA_bio15,Max_Rachis_Length_m,StemHeightBladeLength,HeightOverCanopy
0,dissected,1.0,253.50000,838.999404,11954.749997,498.499749,2.60,7.00,4.290000e+10
1,entire,1.0,247.49798,1596.493658,2335.730828,540.481267,1.32,3.77,1.514001e+10


In [19]:
palmtrait_kissling=pd.read_csv('../data_files/PalmTraits_1.0.txt')
list(palmtrait_kissling.columns)
palmtrait_kissling[['SpecName','Max_Rachis_Length_m']].sort_values(by='Max_Rachis_Length_m',ascending=False).reset_index()

,index,SpecName,Max_Rachis_Length_m
0,2296,Raphia_farinifera,18.5
1,280,Bactris_killipii,15.0
2,2311,Raphia_taedigera,15.0
3,2313,Raphia_vinifera,12.0
4,1957,Oenocarpus_bataua,11.0
...,...,...,...
2552,2517,Veitchia_subdisticha,NaN
2553,2518,Veitchia_vitiensis,NaN
2554,2520,Verschaffeltia_splendida,NaN
2555,2527,Wallichia_nana,NaN


In [20]:
palmtrait_kissling.columns
palmtrait_kissling['MaxStemHeight_m'].max()

170.0

In [21]:
list(intree.columns)
intree[['StemHeightBladeLength','Simard_Pinto_3DGlobalVeg_JGR','HeightOverCanopy']].sort_values(by='HeightOverCanopy',ascending=False).reset_index()

,index,StemHeightBladeLength,Simard_Pinto_3DGlobalVeg_JGR,HeightOverCanopy
0,964,1.772322,0.0,11.764923
1,156,1.708421,0.0,11.699838
2,1460,1.682145,0.0,11.673021
3,1361,1.677151,0.0,11.667920
4,463,1.672098,0.0,11.662758
...,...,...,...,...
2066,2190,1.459392,NaN,NaN
2067,2191,0.000000,NaN,NaN
2068,2192,0.841985,NaN,NaN
2069,2193,0.812913,NaN,NaN


In [22]:
intree[['StemHeightBladeLength','Simard_Pinto_3DGlobalVeg_JGR','HeightOverCanopy']][:10].apply(lambda x: 10**x).sort_values(by='HeightOverCanopy',ascending=False).reset_index()

,index,StemHeightBladeLength,Simard_Pinto_3DGlobalVeg_JGR,HeightOverCanopy
0,1,22.65,1.0,2.165000e+11
1,0,16.50,1.0,1.550000e+11
2,6,16.00,1.0,1.500000e+11
3,8,13.65,1.0,1.265000e+11
4,9,11.42,1.0,1.042000e+11
5,4,2.41,1.0,1.410000e+10
6,2,1.00,1.0,1.000000e+00
7,3,1.00,1.0,1.000000e+00
8,5,1.00,1.0,1.000000e+00
9,7,1.00,1.0,1.000000e+00


In [23]:
# # summary=intree.groupby(by='shape')[variables].median().apply(lambda x: 10**x).reset_index().rename(mapper, axis=1).copy(deep=True)
mapper=dict(zip([x for x in variables],['%s_median'%(x) for x in variables]))
summarymedian=intree.groupby(by='shape')[variables].median().apply(lambda x: 10**x).rename(mapper, axis=1)#.copy(deep=True)
mapper=dict(zip([x for x in variables],['%s_min'%(x) for x in variables]))
summarymin=intree.groupby(by='shape')[variables].min().apply(lambda x: 10**x).rename(mapper, axis=1)#.reset_index()#.copy(deep=True)
mapper=dict(zip([x for x in variables],['%s_max'%(x) for x in variables]))
summarymax=intree.groupby(by='shape')[variables].max().apply(lambda x: 10**x).rename(mapper, axis=1)#.reset_index()#.copy(deep=True)
summary=pd.concat([summarymin,summarymedian,summarymax],axis=1,sort=True)

for col in variables:
    summary.loc[summary.index,'%s_range'%(col)]=summary['%s_max'%(col)]-summary['%s_min'%(col)]
# summary.T.astype('int').to_csv('../data_files/variable_summary-by_shape_20220822.csv',sep='\t')
summary.T.astype('int')

shape,bipinnate,cospalmate,entire,pinnate,variable
CHELSA_ai_min,1,1,1,1,1
CHELSA_bio1_min,186,44,38,17,21
CHELSA_bio4_min,181,109,83,92,136
CHELSA_bio12_min,3402,96,160,12,144
CHELSA_bio15_min,141,108,111,96,106
Max_Rachis_Length_m_min,5,1,1,1,1
StemHeightBladeLength_min,1,1,1,1,1
HeightOverCanopy_min,80000000001,1,1,1,1
CHELSA_ai_median,1,1,1,1,100000
CHELSA_bio1_median,250,211,246,257,252
